In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'compsci596e/dataset/reviews'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/compsci596e/dataset/reviews


In [ ]:
# Installing all dependencies

!python -m spacy download en_core_web_lg
!pip install spacy_universal_sentence_encoder

2022-10-17 22:17:14.160950: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 9.5 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for spacy-universal-sentence-encoder: filename=spacy_universal_sentence_encoder-0.4.5-py3-none-any.whl size=15810 sha256=ebe16f0d66a857c575024c7b29adefbaecff88633a22eea8f662d99efc557c9c
  Stored in directory: /root/.cache/pip/wheels/2c/1a/1d/ace0dfd6761dabc0ad566c3886a0869b0748df0440a7d05b27
Successfully built spacy-universal-sentence-encoder


In [ ]:
import spacy
import spacy_universal_sentence_encoder
nlp = spacy.load('en_core_web_lg')
nlp_use = spacy_universal_sentence_encoder.load_model('en_use_lg')

Downloaded https://tfhub.dev/google/universal-sentence-encoder-large/5, Total size: 577.10MB



In [ ]:
#@title Obtaining noun phrases from each review
#Details:
#1. From each review, we find non-repeated nouns only.
#2. Hence, for each review, we create a dictionary of noun:phrase mapping and store the longest (hence most informative) phrase containing that mapping
import pandas as pd
import spacy

nlp = spacy.load('en_core_web_lg')

all_reviews = pd.read_csv("works_reviews.csv")

all_reviews_noun_adj_pairs = []

for review_ind in all_reviews.index:
  review = all_reviews['review_text'][review_ind]
  
  #Excluding null reviews
  if(type(review) == type('')):
    review = nlp(review)
    noun_to_phrase_dic ={}

    for chunk in review.noun_chunks:
      phrase_words = []
      noun = ''

      # Add only root words of noun / adjective terms of the noun-adj pairs
      for tok in chunk:
          if tok.pos_ == "NOUN":
            noun = tok.text
            phrase_words.append((str(tok.lemma_)).lower())
          if tok.pos_ == "ADJ":
            phrase_words.append((str(tok.lemma_)).lower())

      # Append generated phrase to noun adj pair only if the noun in not null
      if noun:
        if(len(phrase_words) > 0):
          phrase = ' '.join(phrase_words)
          
          # If the noun already exits in the map, add it only if the phrase is longer (so no info is lost)
          # We add each noun phrase once so that we do not add the same noun twice.
          #This will help in clustering as two same nouns of the same review will not get clustered if we decide to go that route
          if ((noun not in noun_to_phrase_dic) or len(noun_to_phrase_dic[noun]) < len(phrase)):
            noun_to_phrase_dic[noun] = phrase
        
    all_reviews_noun_adj_pairs.append(noun_to_phrase_dic)
  
print(all_reviews_noun_adj_pairs)

[{'Fans': 'fan', 'stars': 'star', 'cafe': 'cafe', 'time': 'first time', 'chai': 'chai', 'friends': 'beautiful friend', 'burritos': 'beautiful sandwich burrito', 'order': 'pal order', 'mistake': 'easy mistake', 'pass': 'pass', 'area': 'area'}, {'lunch': 'early lunch', 'child': 'picky child', 'atmosphere': 'atmosphere', 'people': 'people', 'food': 'food', 'props': 'prop', 'gentleman': 'nice gentleman'}, {'place': 'place', 'food': 'food', 'smoothies': 'smoothie', 'smoothie': 'smoothie', 'lots': 'lot', 'fruit': 'fruit', 'cafe': 'nice cafe', 'fun': 'great fun', 'options': 'vegetarian vegan option'}, {'order': 'big group order', 'staff': 'staff', 'experience': 'helpful accommodating pleasant experience', 'Food': 'food'}, {'place': 'place', 'students': 'student', 'people': 'people', 'rest': 'rest', 'coffee': 'coffee', 'tea': 'tea', 'food': 'organic food', 'bread': 'organic farm bread', 'bagel': 'bagel', 'cheese': 'cream cheese', 'salmon': 'salmon', ':)': ':)'}, {'place': 'sandwich place', 'qu

In [ ]:
#@title Find all subphrases containing nouns from the given phrases
# eg - nice cozy room should have all - ['room', 'cozy room', 'nice cozy room'] but not 'nice cozy', 'nice' and 'cozy'
def find_all_subphrases(phrase):
  subphrases = []
  words_in_phrase = phrase.split()
  num_of_words = len(words_in_phrase)

  for subphrase_length in range(1, num_of_words + 1):
    for subphrase_ind in range(0, num_of_words - subphrase_length + 1):
      subphrase = ' '.join(words_in_phrase[subphrase_ind: (subphrase_ind + subphrase_length)])
      
      # Add a subphrase only if it contains a noun
      if(len([np.text for np in nlp(subphrase).noun_chunks]) > 0):
        subphrases.append(subphrase)
  return subphrases

In [ ]:
#@title Find the count of each of the above phrases

phrase_count = {}

# Add a count of all the subphrases
for noun_to_phrase_dic in all_reviews_noun_adj_pairs:

  for phrase in noun_to_phrase_dic.values():
    subphrases = find_all_subphrases(phrase)

    for subphrase in subphrases:
      if subphrase in phrase_count:
        phrase_count[subphrase] += 1
      else:
        phrase_count[subphrase] = 1

print(phrase_count)

{'fan': 1, 'star': 5, 'cafe': 10, 'time': 10, 'first time': 1, 'chai': 1, 'friend': 6, 'beautiful friend': 1, 'sandwich': 30, 'burrito': 1, 'beautiful sandwich': 1, 'sandwich burrito': 1, 'beautiful sandwich burrito': 1, 'pal': 1, 'order': 9, 'pal order': 1, 'mistake': 2, 'easy mistake': 1, 'area': 5, 'lunch': 13, 'early lunch': 1, 'child': 1, 'picky child': 1, 'atmosphere': 11, 'people': 7, 'food': 64, 'prop': 1, 'gentleman': 1, 'nice gentleman': 1, 'place': 32, 'smoothie': 12, 'lot': 5, 'fruit': 1, 'nice cafe': 1, 'fun': 3, 'great fun': 1, 'vegetarian': 2, 'vegan': 3, 'option': 13, 'vegetarian vegan': 1, 'vegan option': 2, 'vegetarian vegan option': 1, 'group': 2, 'big group': 1, 'group order': 1, 'big group order': 1, 'staff': 28, 'experience': 4, 'pleasant experience': 1, 'accommodating pleasant experience': 1, 'helpful accommodating pleasant experience': 1, 'student': 2, 'coffee': 32, 'tea': 1, 'organic food': 2, 'farm': 1, 'bread': 7, 'organic farm': 1, 'farm bread': 1, 'organic 

In [ ]:
#@title Find the sorted order of each of the occurences

sorted_phrases = sorted(phrase_count.items(), key=lambda x: x[1], reverse=True)

print(sorted_phrases)

[('food', 64), ('place', 32), ('coffee', 32), ('sandwich', 30), ('staff', 28), ('bagel', 23), ('service', 18), ('breakfast', 16), ('customer', 14), ('lunch', 13), ('option', 13), ('smoothie', 12), ('cheese', 12), ('menu', 12), ('atmosphere', 11), ('salad', 11), ('price', 11), ('cafe', 10), ('time', 10), ('order', 9), ('seating', 9), ('great food', 8), ('people', 7), ('bread', 7), ('cream', 7), ('cream cheese', 7), ('soup', 7), ('customer service', 7), ('selection', 7), ('friend', 6), ('ingredient', 6), ('outdoor', 6), ('town', 6), ('star', 5), ('area', 5), ('lot', 5), ('table', 5), ('thing', 5), ('friendly staff', 5), ('spot', 5), ('outdoor seating', 5), ('choice', 5), ('restaurant', 5), ('good food', 5), ('day', 5), ('wifi', 5), ('employee', 5), ('great place', 5), ('good coffee', 5), ('experience', 4), ('quality', 4), ('chicken', 4), ('egg', 4), ('kid', 4), ('item', 4), ('environment', 4), ('location', 4), ('fun', 3), ('vegan', 3), ('course', 3), ('bakery', 3), ('chain', 3), ('helpfu

In [ ]:
noun = ''
if noun:
  print('yes')
else:
  print('no')

no
